In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from keras import callbacks
from keras import optimizers
import json
from helper import *
from config import *
from dataloader import *
from model import *

In [2]:
data = DataLoader(config_aide)

Train dataset
Train dataset loaded: 800
Photos: train=800
Loaded: 1000 
Original Vocabulary Size: 3273
Save descriptions in separate file
Descriptions: train=800
Number of training captions  1600
Description Length: 36
preprocessed words 2908 -> 265
Vocab size:  266
Found 1926321 word vectors.


In [3]:
model=ModelImpl(data)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 36)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 36, 99)       26334       input_3[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [4]:
model.train()

8/8 [==============================] - 7s 628ms/step - loss: 5.0473


/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [5]:
model.evaluate()


tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 150, 'reflen': 1909, 'guess': [150, 3, 0, 0], 'correct': [107, 0, 0, 0]}
ratio: 0.07857517024616104
Bleu_1: 0.000
Bleu_2: 0.000
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.023
computing Rouge score...
ROUGE_L: 0.088
computing CIDEr score...
CIDEr: 0.017
computing SPICE score...
SPICE: 0.000
computing WMD score...
WMD: 0.205
results/aide.json
{'Bleu_1': 5.760575780762802e-06, 'Bleu_2': 1.2452570450235878e-13, 'Bleu_3': 5.002999545914669e-13, 'Bleu_4': 1.0028043091238805e-12, 'METEOR': 0.02332086163681936, 'ROUGE_L': 0.08786804535050532, 'CIDEr': 0.016848116084526923, 'SPICE': 0.0, 'WMD': 0.20470539763570714}
